In [1]:
import pygetwindow as pgw
from PIL import ImageGrab,Image
import pyautogui as pag
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
import time
from datetime import datetime
import re
import pytesseract as tess
import subprocess

In [2]:
def display_img(img,cmap=None):
    fig = plt.figure(figsize=(15*win_width/win_height,10))
    ax = fig.add_subplot(111)
    ax.imshow(img,cmap)
    

In [3]:
# https://stackoverflow.com/questions/4993082/how-can-i-sharpen-an-image-in-opencv
def unsharp_mask(image, kernel_size=(5, 5), sigma=1.0, amount=1.0, threshold=0):
    """Return a sharpened version of the image, using an unsharp mask."""
    blurred = cv.GaussianBlur(image, kernel_size, sigma)
    sharpened = float(amount + 1) * image - float(amount) * blurred
    sharpened = np.maximum(sharpened, np.zeros(sharpened.shape))
    sharpened = np.minimum(sharpened, 255 * np.ones(sharpened.shape))
    sharpened = sharpened.round().astype(np.uint8)
    if threshold > 0:
        low_contrast_mask = np.absolute(image - blurred) < threshold
        np.copyto(sharpened, image, where=low_contrast_mask)
    return sharpened

In [4]:
# https://stackoverflow.com/questions/2846947/get-screenshot-on-windows-with-python
def get_app_window():
    ss = pag.screenshot()
    ss = np.asarray(ss)
    ss = unsharp_mask(ss)
    roi = ss[:dim[1],:dim[0],:]
    return roi

In [5]:
def is_rune(pix):
    red_channel = pix[0,0,0]
    green_channel = pix[0,0,1]
    blue_channel = pix[0,0,2]
    if red_channel == 255 and green_channel == 255:
        return 1
    else:
        return 0

In [6]:
# """
# refill button:
# 160 : 175 , 513 : 530, :
# """
# pag.click(x=np.random.randint(513,531),
#           y=np.random.randint(160,176))

# time.sleep(1)

# """
# charging method button:
# 336 : 370 , 300 : 400, :
# """
# pag.click(x=np.random.randint(300,400),
#           y=np.random.randint(336,370))

In [7]:
dict_pos_reward = {

'px_0' : (239,564),
'px_1' : (239,633),
'px_2' : (239,702),
'px_3' : (239,771),
'px_4' : (308,564),
'px_5' : (308,633),
'px_6' : (308,702),
'px_7' : (308,771),
'px_8' : (377,564),
'px_9' : (377,633)
    
}

In [8]:
def awake_screen():
    """
    the function awakes screen since SW automatically sleeps after a while
    """
    pt1 = (np.random.randint(200,300),np.random.randint(200,300))
    pt2 = (np.random.randint(400,500),np.random.randint(300,400))
    pag.moveTo(pt1[0],pt1[1])
    pag.dragTo(pt2[0], pt2[1], button='left',duration = 1,tween= pag.easeInOutQuad)
    time.sleep(1)

def click_sell_rune():
    # click sell button
    pag.click(x=np.random.randint(330,400) ,
              y=np.random.randint(400,430))
    time.sleep(np.random.randint(2,4))
    # click yes 
    pag.click(x=np.random.randint(320,400),
              y=np.random.randint(330,360))
    time.sleep(np.random.randint(2,4))
    
    pic = get_app_window()
    ret,thres = threshold(pic[200:240,410:560])
    text = tess.image_to_string(thres).split('\n')
    text.remove('\x0c')
    if 'LEGEND' in text[0].upper(): # com2us put another notification when selling legend runes. we need to bypass it
        print('sell LEGEND!')
        pag.click(x=np.random.randint(320,400),
                  y=np.random.randint(330,360))
        time.sleep(np.random.randint(2,4))

def click_sell_selected():
    #click Sell Selected
    pag.click(x=np.random.randint(720,795),y=np.random.randint(470,490))
    time.sleep(np.random.randint(1,3))
    #click Sell selected
    pag.click(x=np.random.randint(620,680),y=np.random.randint(470,490))
    time.sleep(np.random.randint(1,3))
    
    pic = get_app_window()
    ret,thres = threshold(pic[220:260,250:600])
    list_noti = tess.image_to_string(thres).split('\n')
    list_noti.remove('\x0c')
    
    if "select" in list_noti[0]:
        #You didnt selected items to sell:
        #Click OK
        pag.click(x=np.random.randint(400,460),y=np.random.randint(330,360))
        time.sleep(np.random.randint(1,3))
        #Click cancel pic[470:490,725:800]
        pag.click(x=np.random.randint(725,800),y=np.random.randint(470,490))
        time.sleep(np.random.randint(1,3))
    else:    
        #sell successfully
        pag.click(x=np.random.randint(320,400),y=np.random.randint(330,360))
        time.sleep(np.random.randint(4,8))

In [9]:
def threshold(pic):
    roi = pic.copy()
    roi = cv.cvtColor(roi,cv.COLOR_BGR2GRAY)
    ret,thres = cv.threshold(roi,127,255,cv.THRESH_BINARY_INV)
    return ret,thres

In [10]:
# class A:
#     def __init__(self):
#         self.name = 'rune A'
#         self.type = 'legend'
#         self.prop_sub = {'a':1,'b':2}
        
#     def __str__(self):
#         list_prop = [
#             f"Rune name: {self.name}",
#             f"Rune type: {self.type}",
#             f"Rune prop sub: {self.prop_sub}"
#         ]
#         resp= "\n".join(list_prop)
#         return resp
#     def self_print(self):
#         print(self)
        
# t = A()
# t.self_print()

In [11]:
class Rune:
    def __init__(self,pic):
        self.pic = pic
        self.name,self.slot = self.get_rune_name()
        self.type = self.get_rune_type()
        self.prop_main,self.prop_innate,self.prop_sub = self.get_rune_prop()
        
    def __str__(self):
        list_prop = [
            f"Rune name: {self.name}",
            f"Rune type: {self.type}",
            f"Rune prop_main: {self.prop_main}",
            f"Rune prop_innate: {self.prop_innate}",
            f"Rune prop_sub: {self.prop_sub}"
        ]
        resp= "\n".join(list_prop)
        return resp
        
    def threshold(self,pic):
        ret,thres = threshold(pic)
        return thres
    
    def get_rune_name(self):       
        thres = self.threshold(self.pic[135:175,315:550,:])
        rune_name_str = tess.image_to_string(thres).split('\n')[0]
        rune_name = re.sub('[^a-zA-Z| ]','',rune_name_str).strip()
        rune_slot = re.sub('[\D]','',rune_name_str)
        return rune_name,rune_slot
    
    def get_rune_type(self):  
        thres = self.threshold(self.pic[185:204,520:610,:])
        res = tess.image_to_string(thres).split('\n')
        res.remove('\x0c')
        return res[0]
    
    def get_rune_prop(self):
        thres = self.threshold(self.pic[180:230,330:510,:])
        list_prop = tess.image_to_string(thres).split('\n')
        list_prop.remove('\x0c')
        rune_prop_main = (re.sub('[^a-zA-Z|]','',list_prop[0]),re.sub('[^\d|%]','',list_prop[0]))
        if len(list_prop) == 2:       
            rune_prop_innate = (re.sub('[^a-zA-Z|]','',list_prop[1]),re.sub('[^\d|%]','',list_prop[1]))
        else:
            rune_prop_innate = None
        
        thres = self.threshold(self.pic[240:340,250:600,:])
        list_prop_sub = tess.image_to_string(thres).split('\n')
        list_prop_sub.remove('\x0c')
        rune_prop_sub = {}
        for sub in list_prop_sub:
            #rune_prop_sub.append((re.sub('[^a-zA-Z|]','',sub),re.sub('[^\d|%]','',sub)))
            rune_prop_sub[re.sub('[^a-zA-Z|]','',sub)] = re.sub('[^\d|%]','',sub)
        return rune_prop_main,rune_prop_innate,rune_prop_sub
    
    def has_speed_sub(self):
        flag = 0
        for sub in self.prop_sub.keys():
            if 'SPD' in sub:
                flag = 1
        return flag

    def strategy_gb12(self):
        if 'Resistance' in self.prop_main[0]:
            print('SOLD as Resistance in prop main!')
            click_sell_rune() 
        elif self.type.lower() in ['magic','blue','normal']:
            print('SOLD as Magic, Blue, Normal')
            click_sell_rune()             
        elif self.slot in ['4','6'] and '%' not in self.prop_main[1]:
            print('SOLD as % not in slot 4,6!')
            click_sell_rune() 
        elif self.slot in ['2'] and 'SPD' not in self.prop_main[0] and '%' not in self.prop_main[1]:
            print('SOLD as SPD and % not in slot 2!')
            click_sell_rune() 
        elif self.has_speed_sub() == 0:
            print('SOLD as has no speed sub!')
            click_sell_rune()
        else:
            pag.click(x=np.random.randint(597,609),
                    y=np.random.randint(146,159)) 
            time.sleep(1)
        
    def strategy_db12(self):
        if 'Resistance' in self.prop_main[0]:
            print('SOLD as Resistance in prop main!')
            click_sell_rune() 
        elif self.type.lower() in ['magic','blue','normal']:
            print('SOLD as Magic, Blue, Normal')
            click_sell_rune()             
        elif self.slot in ['4','6'] and '%' not in self.prop_main[1]:
            print('SOLD as % not in slot 4,6!')
            click_sell_rune() 
        elif self.slot in ['2'] and 'SPD' not in self.prop_main[0] and '%' not in self.prop_main[1]:
            print('SOLD as SPD and % not in slot 2!')
            click_sell_rune() 
        elif not (('Violent' in self.name and 'Legend' in self.type) 
                  or ('Violent' in self.name and 'SPD' in self.prop_main[0]) 
                  or self.has_speed_sub() == 1):
            print('SOLD as not Legend Violent and has no speed sub!')
            print(self)
            click_sell_rune()
        # elif 'Violent' not in self.name and self.has_speed_sub() == 0:
        #     print('SOLD as not Violent and has no speed sub!')
        #     click_sell_rune()
        else:
#             print(self)
            pag.click(x=np.random.randint(597,609),
                    y=np.random.randint(146,159)) 
            time.sleep(1) 

    def strategy_nb12(self):
        if 'Resistance' in self.prop_main[0]:
            print('SOLD as Resistance in prop main!')
            click_sell_rune() 
        elif self.type.lower() in ['magic','blue','normal']:
            print('SOLD as Magic, Blue, Normal')
            click_sell_rune()             
        elif self.slot in ['4','6'] and '%' not in self.prop_main[1]:
            print('SOLD as % not in slot 4,6!')
            click_sell_rune() 
        elif self.slot in ['2'] and 'SPD' not in self.prop_main[0] and '%' not in self.prop_main[1]:
            print('SOLD as SPD and % not in slot 2!')
            click_sell_rune() 
        elif 'Will' not in self.name and 'Rage' not in self.name and self.has_speed_sub() == 0:
            print('SOLD as not Will/Rage and has no speed sub!')
            click_sell_rune()
        else:
            print(self)
            pag.click(x=np.random.randint(597,609),
                    y=np.random.randint(146,159)) 
            time.sleep(1)
        

In [12]:

def check_blue_runes(pic):
    loc_blue_rune = []
    loc_not_blue_rune = []
    for idx,pos in dict_pos_reward.items():
        y = pos[0]
        x = pos[1]
        
        if is_rune(pic[y:y+1,x:x+1,:]) == 0:
            pass
            # print(f'{idx} is NOT a rune')
        else:
            mean_red_channel = pic[y+15:y+20,x:x+1,:][:,:,0].mean()
            if mean_red_channel < 100:
                #print(f'{idx} is blue rune: mean_red_channel = {mean_red_channel}')
                loc_blue_rune.append(pos)
            else:
                #print(f'{idx} is NOT blue rune: mean_red_channel = {mean_red_channel}')
                loc_not_blue_rune.append(pos)
    return loc_blue_rune,loc_not_blue_rune



def sell_blue_runes(list_loc):
    for pos in list_loc:
        y_click = np.random.randint(pos[0],pos[0]+40)
        x_click = np.random.randint(pos[1],pos[1]+40)
#         print(y_click,x_click)
        # click blue runes
        pag.click(x=x_click,
                  y=y_click)
        time.sleep(np.random.randint(2,4))
        click_sell_rune()

def sell_not_blue_runes(list_loc):
    for pos in list_loc:
        y_click = np.random.randint(pos[0],pos[0]+40)
        x_click = np.random.randint(pos[1],pos[1]+40)
        # click hero and legend runes
        pag.click(x=x_click,
                  y=y_click)
        time.sleep(np.random.randint(2,4))
        
        pic = get_app_window()
        rune = Rune(pic)
        
        #rune.strategy_nb12()
        rune.strategy_db12()
        #rune.strategy_gb12()
    
def click_replay():
    pag.click(x=np.random.randint(420,500),
              y=np.random.randint(460,480))
    time.sleep(np.random.randint(2,4))
    pag.click(x=np.random.randint(675,800),
              y=np.random.randint(380,410))

In [13]:
def compare(target,source,method=cv.TM_CCOEFF):
    # The Full Image to Search
    src_img = cv.cvtColor(source, cv.COLOR_BGR2RGB)

    # The Template to Match
    target_img = cv.cvtColor(target, cv.COLOR_BGR2RGB)

    # Apply template Matching with the method
    res = cv.matchTemplate(src_img,target_img,method)

    # Grab the Max and Min values, plus their locations
    min_val, max_val, min_loc, max_loc = cv.minMaxLoc(res)
    top_left = max_loc
    
    return top_left

def is_finished_repeat_battle():
    pic = get_app_window()
    roi = pic[385:415,80:500,:].copy()
    roi = cv.cvtColor(roi,cv.COLOR_BGR2GRAY)
    ret,thres = cv.threshold(roi,127,255,cv.THRESH_BINARY_INV)
    try:
        res = tess.image_to_string(thres).split('\n')[0]
    except:
        res == 'ERROR'
        return 0

    if 'COMPLETED' in res.upper() or 'ENDED' in res.upper() :
        return 1
    elif 'LOST' in res.upper():
        print(f"LOST!")
        return 1
    else:
        return 0

In [14]:
def is_finised_toa():
    pic = get_app_window()
    roi = pic[100:160,270:600,:].copy()
    roi = cv.cvtColor(roi,cv.COLOR_BGR2GRAY)
    ret,thres = cv.threshold(roi,127,255,cv.THRESH_BINARY_INV)
    try:
        res = tess.image_to_string(thres).split('\n')[0]
    except:
        res == 'ERROR'
    if res == 'VICTORY':
        return 1
    elif res == 'DEFEATED':
        return 2
    else:
        return 0

In [15]:
win = pgw.getWindowsWithTitle('BlueStacks')[0]
win.maximize()
win.moveTo(0,0)
win.size = (944,540)
dim = win.size
win_width = dim[0]
win_height = dim[1]
dim

Size(width=944, height=540)

In [16]:
pag.FAILSAFE = False

In [17]:
### AUTO FARM RUNES
def auto_farm_runes(_from = None, _to = None):
    while True:
        awake_screen()
        time.sleep(1)
        pic = get_app_window()
        try:
            loc_blue_runes,loc_not_blue_rune = check_blue_runes(pic)
            sell_blue_runes(loc_blue_runes)
            sell_not_blue_runes(loc_not_blue_rune)
        except:
            pass

        if _from is None and _to is None:
            pass
        elif datetime.now().hour >= _from and datetime.now().hour < _to:
            subprocess.run(["shutdown", "-s"])
            break

        if is_finished_repeat_battle() == 1:
            # due to some bugs,we click on the X button of runes frame before click replay
            pag.click(x=np.random.randint(597,609),
                    y=np.random.randint(146,159)) 
            time.sleep(1)
            print('FINISHED!')
            click_replay()
        else:
            time.sleep(5)

In [18]:
### AUTO FARM ARTIFACTS
def auto_farm_artifact(_from = None, _to = None):
    while True:
        
        if _from is None and _to is None:
            pass
        elif datetime.now().hour >= _from and datetime.now().hour < _to:
            subprocess.run(["shutdown", "-s"])
            break
            
        awake_screen()
        time.sleep(1)
        pic = get_app_window()

        if is_finished_repeat_battle() == 1:
            # due to some bugs,we click on the X button of runes frame before click replay
            pag.click(x=np.random.randint(597,609),
                    y=np.random.randint(146,159)) 
            click_sell_selected()
            time.sleep(3)
            print('FINISHED!')
            click_replay()
        else:
            time.sleep(5)

In [19]:
### AUTO TOA
def auto_toa():
    while True:
        flag_finished = is_finised_toa()
        if flag_finished == 1: # victory
            
            # click mouse 2 times to proceed to reward screen
            pag.click(x=np.random.randint(420,500),y=np.random.randint(460,480))
            time.sleep(np.random.randint(1,3))
            pag.click(x=np.random.randint(420,500),y=np.random.randint(460,480))
            
            # click OK button
            time.sleep(np.random.randint(2,4))
            pag.click(x=np.random.randint(400,470),y=np.random.randint(410,440))
            
            # click Next button
            time.sleep(np.random.randint(1,3))
            pag.click(x=np.random.randint(200,380),y=np.random.randint(310,320))
            
            # click start battle
            time.sleep(np.random.randint(2,4))
            pag.click(x=np.random.randint(690,760),y=np.random.randint(370,420))
            
        elif flag_finished == 2:
            print('STOP program due to DEFEAT')
            break
        else:
            pass

In [21]:
# auto_toa()
# auto_farm_runes()
auto_farm_artifact()

FINISHED!
FINISHED!
FINISHED!
FINISHED!
FINISHED!
FINISHED!
FINISHED!
FINISHED!
FINISHED!


KeyboardInterrupt: 

In [ ]:
ret,thres = threshold(pic[200:240,410:560])
text = tess.image_to_string(thres).split('\n')
text.remove('\x0c')
text
# if 'LEGEND' in text[0].upper(): # com2us put another notification when selling legend runes. we need to bypass it
#     print('sell LEGEND!')
#     pag.click(x=np.random.randint(320,400),
#               y=np.random.randint(330,360))
#     time.sleep(np.random.randint(2,4))

In [ ]:
display_img(pic)

In [ ]:
    # click yes 
    pag.click(x=np.random.randint(320,400),
              y=np.random.randint(330,360))

In [ ]:
dim

In [ ]:
pic = get_app_window()

In [ ]:
rune = Rune(pic)

In [ ]:
rune.slot

In [ ]:
rune.prop_main

In [ ]:
display_img(pic)

In [ ]:
rune.type

In [ ]:
class RuleExecutor:
    list_dungeon = ['NB12','DB12','GB12']
    
    def __init__(self,Rune,dungeon):
        self.rune = Rune
        self.dungeon = dungeon
        self.is_valid_dungeon()
    
    def is_valid_dungeon(self):
        if self.dungeon in self.list_dungeon:
            pass
        else:
            raise Exception(f"Valid dungeon must be in {self.list_dungeon}")            
    
    def is_Violent(self): return 'Violent' in self.rune.name
    def is_Will(self): return 'Will' in self.rune.name
    def is_Swift(self): return 'Swift' in self.rune.name
    def is_Despair(self): return 'Despair' in self.rune.name
    def is_Rage(self): return 'Rage' in self.rune.name
    
    def is_type_legend(self): return 'Legend' in self.rune.type
    def is_type_hero(self): return 'Hero' in self.rune.type 
    
    def is_slot_2(self): return '2' in self.rune.slot
    def is_slot_4(self): return '4' in self.rune.slot
    def is_slot_6(self): return '6' in self.rune.slot
    
    def is_SPD_main_prop(self): return 'SPD' in self.rune.prop_main[0]
    
    def has_pct_increase(self): return '%' in self.rune.prop_main[1]
    
    def has_speed_sub(self):
        for sub in self.rune.prop_sub.keys():
            return True if 'SPD' in sub else False
        
    def DB12_strategy():
        if self.is_Violent() \
            or self.has_speed_sub() \
            or ((self.is_type_legend() or self.is_type_hero) \
                and self.is_SPD_main_prop())
            or ((self.is_type_legend() or self.is_type_hero) \
                and (self.is_slot_4() or self.is_slot_6()) \
                and self.has_speed_sub()):
            pass
        

In [ ]:
True and False or True and False

In [ ]:
(True and False) or (True and False)

In [ ]:
0self.name,self.slot = self.get_rune_name()
self.type = self.get_rune_type()
self.prop_main,self.prop_innate,self.prop_sub = self.get_rune_prop()

In [ ]:
RE = RuleExecutor(rune,'nb')

In [ ]:
RE.has_speed_sub()

In [ ]:
True if 1 < 0  else False

In [ ]:
if 'Resistance' in self.prop_main[0]:
    print('SOLD as Resistance in prop main!')
#     click_sell_rune() 
elif self.type.lower() in ['magic','blue','normal']:
    print('SOLD as Magic, Blue, Normal')
#     click_sell_rune()             
elif self.slot in ['4','6'] and '%' not in self.prop_main[1]:
    print('SOLD as % not in slot 4,6!')
#     click_sell_rune() 
elif self.slot in ['2'] and 'SPD' not in self.prop_main[0] and '%' not in self.prop_main[1]:
    print('SOLD as SPD and % not in slot 2!')
#     click_sell_rune() 
elif 'Will' not in self.name and 'Rage' not in self.name and self.has_speed_sub() == 0:
    print('SOLD as not Will/Rage and has no speed sub!')
#     click_sell_rune()

In [ ]:
ret,thres = threshold(pic[200:240,410:560])
text = tess.image_to_string(thres).split('\n')
text.remove('\x0c')
text[0]

In [ ]:
ret,thres = threshold(pic[220:260,250:600])
list_noti = tess.image_to_string(thres).split('\n')
list_noti.remove('\x0c')
list_noti

In [ ]:
rune = Rune(pic)

In [ ]:
rune.prop_sub

In [ ]:
rune.has_speed_sub()

In [ ]:
if rune.has_speed_sub():
    pag.click(x=np.random.randint(450,550),y=np.random.randint(400,420))

In [ ]:
pic_1 = get_app_window()

In [ ]:
display_img(pic_1[230:350,480:680])

In [ ]:
ret,thres = threshold(pic_1[230:350,480:680])
list_prop_sub = tess.image_to_string(thres).split('\n')
list_prop_sub.remove('\x0c')

In [ ]:
list_prop_sub

In [ ]:
rune_prop_sub = {}
for sub in list_prop_sub:
    #rune_prop_sub.append((re.sub('[^a-zA-Z|]','',sub),re.sub('[^\d|%]','',sub)))
    rune_prop_sub[re.sub('[^a-zA-Z|]','',sub)] = re.sub('[^\d|%]','',sub)

In [ ]:
rune_prop_sub

In [ ]:
datetime(2022,4,14,13,45,56).hour